In [19]:
import moresim as ms
import numpy as np
import moresim.MC as msc
import ase 
import ase.io as aio
from importlib import reload
import matplotlib.pyplot as plt

In the following there is a quick demonstration of how to set up simulations with the moresim module

## MC simulation

Here we construct a MC simulation of a Lennard-Jones gas

To contruct a Monte Carlo simulation we need:
    - A system
    
    - A set of MC moves
    
    - An energy function
    

### System

All systems in this module consist of object of the class ase.Atoms


In [23]:
init_state = ase.Atoms(['He'] * 40, np.random.uniform(low=0, high=5, size=[40,3]))

### Set of MC moves

The MC moves should be contained in a list. Here we use random particle moves implemented in the module

First we need a random number generator

In [24]:
rand_num_gen = msc.RandNumGen.GaussianVar(loc=0, var=0.05)

Then we contruct the set of MC moves:

In [26]:
mc_moves = [msc.Moves.RandomParticleMove(rand_num_gen=rand_num_gen)]

### Energy Function

The energy function should take a ase.Atoms object and return the potential energy

Here we use a LJ potential

In [28]:
lj_pot_helium = ms.EnergyCalculators.Lennard_Jones(sigma=1, epsilon=1)

In [7]:
ef = lj_pot_helium.energy

### Setting up simulation

We construct the simulation with the setting we just defined

In [29]:
sim1 = ms.Simulations.MCSimulation(energy_func=lj_pot_helium.energy, temperature=300,
                                   move_list=mc_moves)

and we run the simulation 

In [32]:
results = sim1.run(states[0], steps=100, stride=10, return_last=True)

The results are a list containing [A trajectory, the final state, the time spent in seconds]

In [33]:
res

 Atoms(symbols='He40', pbc=False),
 0.0005125308036804199]

We can write the trajectry in a file as

In [34]:
res[0].flush('test_trj_lj.xyz')

## T-RE Simulation

Here we construct a basic replica exchange simulation

We will use a set of MC simulations like the one we constructed above at different temperatures

In [36]:
temperatures = [300, 350, 450, 600]
num_reps = len(temperatures)

In [15]:
rep_ex_simulations = [ms.Simulations.MCSimulation(energy_func=lj_pot_helium.energy, 
                                                  temperature=temp, 
#                                                   atoms=states[0].get_chemical_symbols(),
                                                  move_list=mc_moves) for temp in temperatures]

We generate the initial states

In [37]:
init_states = [init_state for temp in temperatures]

Create the replica exchange simulation

In [38]:
rep_ex = ms.Simulations.ReplicaExchangeSimulation(num_reps=num_reps,
                                      simulations=rep_ex_simulations,
                                      init_states=init_states, stride=10,
                                      rep_steps=20)

/home/fabregat/work/hres/test_library/moresim/utils.py:11: RuntimeWarning: overflow encountered in exp
  prob = min([1, np.exp(- weight_new + weight_old)])


and run it

In [39]:
number_of_exchanges = 10

rep_ex.run(number_of_exchanges)

rep_ex.run() will generate the structure and energy trajectory files. It will also create a file called exchange.txt where the exchange probabilities are written.

## Construct a reservoir

Here we show how to replace one of the MC simulations by a reservoir. Each reservoir structure should be placed in a directory and be called struct_ + 5 leading zeros, e.g. directory/struct_00000.xyz

If the directory was called 'structures' then

In [41]:
structures_folder='./structures'

The energies should be stored in a list or array

In [45]:
reserv_energies = []

In [48]:
reservoir = ms.Simulations.Reservoir(structures_folder=structures_folder,
                         energies=reserv_energies,
                         temperature=300, energy_func=lj_pot_helium)

The reservoir structures should follow the canonical distribution according to the assigned potential energy.